In [1]:
import note_seq
from note_seq import drums_lib
from note_seq import midi_io
from note_seq.protobuf import music_pb2
from note_seq import sequences_lib
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
from helpers import *
import numpy as np
from classes import *
import random
import time

In [2]:
## drumtrack to sequence to midi

# lol=drums[0:64].to_sequence()
# # print(lol)
# midi_io.sequence_proto_to_midi_file(lol,'test.midi')

In [4]:
enc = drums_encoder_decoder.MultiDrumOneHotEncoding()

model_path = "models/lstmcyclic2561e-4_de.pth"
net = lstmd()
# net = bigger_v_decoder()

model_path2 = "models/lstmcyclic2561e-4_en.pth"
net2 = lstmm()
# net2 = conv_encoder()

# model_path = "models/autoencode.pth"
# net = ae()

# model_path = "models/rnn.pth"
# net = rnn()

net2.load_state_dict(torch.load(model_path2))
net.load_state_dict(torch.load(model_path))

# net

<All keys matched successfully>

In [4]:
##input array making
## pick midi
current = os.getcwd()
path = os.path.join(current, "data") 
path = os.path.join(path, "drummer1") 
## choose session
path = os.path.join(path, "eval_session") 
## choose midi
midi_path = os.path.join(path, random.choice(os.listdir(path))) 
in_array = midi_to_input(midi_path)

    
in_tensor = torch.tensor(np.expand_dims(in_array,0)).to(device='cpu', non_blocking=True).float().unsqueeze(0)
mean,var = net2(in_tensor)
print(mean)
print(var)

RuntimeError: input.size(-1) must be equal to input_size. Expected 72, got 576

In [12]:
# noise = torch.randn(1,32)
noise = torch.normal(torch.zeros(1,1,256), std=2)
# noise = torch.tensor([[3.25,0.05,30.08,0.91,11.42,0.3,90.45,5.7]]).float()
# music = np.rint(net.decode(noise).detach().numpy()).astype(int)
music = np.rint(net.forward(noise,num=16).detach().numpy()).astype(int)[0]
music = np.expand_dims(music.flatten(),0)

drums = drums_lib.DrumTrack()
print(music.shape)
for i in range(1152//9):
    target = music[0,i*9:(i+1)*9]
#     print(target)
    res = 0
    for ele in target: 
        res = (res << 1) | ele
#     print(res)
    drums.append(enc.decode_event(res))
lol=drums.to_sequence()
midi_io.sequence_proto_to_midi_file(lol,'test.midi')
    
        

(1, 1152)


In [ ]:
note = torch.zeros(1,9)
hidden = torch.randn(1,8)
drums = drums_lib.DrumTrack()
i=0
while not torch.equal(note,torch.ones(9).float()):
    note_array =  np.rint(note[0].detach().numpy()).astype(int)
    res = 0
    for ele in note_array: 
        res = (res << 1) | ele
    print(note_array)
    drums.append(enc.decode_event(res))
    note, hidden = net(note,hidden)
    i+=1
    
    
    